<a href="https://colab.research.google.com/github/costaljorge/Interpolar_Extrapolar/blob/main/Interpolar_Extrapolar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sklearn

In [9]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta

# Suponha que você tenha um DataFrame chamado 'dados' com as colunas 'Data' e 'Valor'

# Carregando os dados
dados = pd.read_excel('/content/Temperatura.xlsx')  # Substitua 'seu_dataset.csv' pelo caminho do seu arquivo CSV

# Convertendo a coluna 'Data' para o tipo datetime
dados['Data'] = pd.to_datetime(dados['Data'])

# Criando uma nova coluna de datas para os próximos 10 dias
data_maxima = dados['Data'].max()
proximos_10_dias = [data_maxima + timedelta(days=i) for i in range(1, 11)]

# Convertendo as datas para o formato numérico (dias desde a data mínima)
data_minima = dados['Data'].min()
dados['Dias'] = (dados['Data'] - data_minima).dt.days

# Preparando os dados para treinamento do modelo
X = dados[['Dias']]
y = dados['Valor']

# Instanciando e treinando o modelo de regressão linear
modelo = LinearRegression()
modelo.fit(X, y)

# Criando um DataFrame com as datas para os próximos 10 dias
df_predicao = pd.DataFrame(proximos_10_dias, columns=['Data'])
df_predicao['Dias'] = (df_predicao['Data'] - data_minima).dt.days

# Fazendo a previsão para os próximos 10 dias
previsao = modelo.predict(df_predicao[['Dias']])

# Criando um DataFrame com as datas e as previsões
df_previsao_final = pd.DataFrame({'Data': proximos_10_dias, 'Previsao': previsao})

# Calculando o erro da previsão
# Para este exemplo, calcularei o erro médio absoluto
erro = np.mean(np.abs(previsao - y.mean()))

print("Previsão para os próximos 10 dias:")
print(df_previsao_final)
print("Erro da previsão:", erro)


Previsão para os próximos 10 dias:
        Data   Previsao
0 2023-12-31  25.993591
1 2024-01-01  25.996097
2 2024-01-02  25.998604
3 2024-01-03  26.001111
4 2024-01-04  26.003618
5 2024-01-05  26.006125
6 2024-01-06  26.008632
7 2024-01-07  26.011138
8 2024-01-08  26.013645
9 2024-01-09  26.016152
Erro da previsão: 0.47707566210255764


## Statsmodels

In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta

# Carregando os dados do arquivo Excel
dados = pd.read_excel('/content/Temperatura.xlsx')  # Substitua 'seu_dataset.xlsx' pelo caminho do seu arquivo Excel

# Convertendo a coluna 'Data' para o tipo datetime
dados['Data'] = pd.to_datetime(dados['Data'])

# Criando uma nova coluna de datas para os próximos 10 dias
data_maxima = dados['Data'].max()
proximos_10_dias = [data_maxima + timedelta(days=i) for i in range(1, 11)]

# Convertendo as datas para o formato numérico (dias desde a data mínima)
data_minima = dados['Data'].min()
dados['Dias'] = (dados['Data'] - data_minima).dt.days

# Dividindo os dados em variáveis independentes (X) e variável dependente (y)
X = sm.add_constant(dados['Dias'])  # Adicionando uma coluna de interceptação
y = dados['Valor']

# Ajustando o modelo
modelo = sm.OLS(y, X).fit()

# Criando um DataFrame com as datas para os próximos 10 dias
df_predicao = pd.DataFrame(proximos_10_dias, columns=['Data'])
df_predicao['Dias'] = (df_predicao['Data'] - data_minima).dt.days
X_predicao = sm.add_constant(df_predicao['Dias'])  # Adicionando uma coluna de interceptação

# Fazendo a previsão para os próximos 10 dias
previsao = modelo.predict(X_predicao)

# Calculando o erro da previsão
# Para este exemplo, calcularei o erro médio absoluto
erro = np.mean(np.abs(previsao - y.mean()))

print("Previsão para os próximos 10 dias:")
print(pd.DataFrame({'Data': proximos_10_dias, 'Previsao': previsao}))
print("Erro da previsão:", erro)


Previsão para os próximos 10 dias:
        Data   Previsao
0 2023-12-31  25.993591
1 2024-01-01  25.996097
2 2024-01-02  25.998604
3 2024-01-03  26.001111
4 2024-01-04  26.003618
5 2024-01-05  26.006125
6 2024-01-06  26.008632
7 2024-01-07  26.011138
8 2024-01-08  26.013645
9 2024-01-09  26.016152
Erro da previsão: 0.4770756621025736


##LSTM

In [ ]:
!pip install tensorflow

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta

# Carregando os dados do arquivo Excel
dados = pd.read_excel('/content/Temperatura.xlsx')  # Substitua 'seu_dataset.xlsx' pelo caminho do seu arquivo Excel

# Convertendo a coluna 'Data' para o tipo datetime
dados['Data'] = pd.to_datetime(dados['Data'])

# Usaremos apenas a coluna 'Valor' para a previsão
valores = dados['Valor'].values.reshape(-1, 1)

# Normalizando os dados entre 0 e 1
scaler = MinMaxScaler()
valores_normalizados = scaler.fit_transform(valores)

# Definindo a função para criar os dados de entrada e saída
def criar_dataset(dados, passos_de_tempo):
    X, y = [], []
    for i in range(len(dados)-passos_de_tempo):
        X.append(dados[i:(i+passos_de_tempo), 0])
        y.append(dados[i + passos_de_tempo, 0])
    return np.array(X), np.array(y)

# Definindo o número de passos de tempo para a previsão
passos_de_tempo = 10  # Prever os próximos 10 dias

# Criando os dados de entrada e saída
X, y = criar_dataset(valores_normalizados, passos_de_tempo)

# Redimensionando os dados para a entrada da LSTM [amostras, passos de tempo, características]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Criando o modelo LSTM
modelo = Sequential()
modelo.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
modelo.add(LSTM(units=50))
modelo.add(Dense(units=1))

# Compilando o modelo
modelo.compile(optimizer='adam', loss='mean_squared_error')

# Treinando o modelo
modelo.fit(X, y, epochs=100, batch_size=32)

# Fazendo previsões para os próximos 10 dias
ultimos_dados = valores_normalizados[-passos_de_tempo:]
previsoes = []
for i in range(passos_de_tempo):
    X_predicao = ultimos_dados[-passos_de_tempo:].reshape(1, passos_de_tempo, 1)
    previsao = modelo.predict(X_predicao)[0]
    previsoes.append(previsao)
    ultimos_dados = np.append(ultimos_dados, previsao.reshape(-1,1), axis=0)

# Invertendo a escala dos dados para obter os valores reais
previsoes = scaler.inverse_transform(previsoes)

# Criando datas para os próximos 10 dias
data_maxima = dados['Data'].max()
proximos_10_dias = [data_maxima + timedelta(days=i) for i in range(1, 11)]

# Criando um DataFrame com as datas e as previsões
df_previsao_final = pd.DataFrame({'Data': proximos_10_dias, 'Previsao': previsoes.flatten()})

# Calculando o erro da previsão
# Para este exemplo, calcularei o erro médio absoluto
erro = np.mean(np.abs(previsoes - dados['Valor'].values[-1]))

print("Previsão para os próximos 10 dias:")
print(df_previsao_final)
print("Erro da previsão:", erro)


Epoch 1/100
6/6 [==============================] - 4s 11ms/step - loss: 0.2028
Epoch 2/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0926
Epoch 3/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0950
Epoch 4/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0850
Epoch 5/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0874
Epoch 6/100
6/6 [==============================] - 0s 12ms/step - loss: 0.0830
Epoch 7/100
6/6 [==============================] - 0s 12ms/step - loss: 0.0850
Epoch 8/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0835
Epoch 9/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0834
Epoch 10/100
6/6 [==============================] - 0s 12ms/step - loss: 0.0828
Epoch 11/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0827
Epoch 12/100
6/6 [==============================] - 0s 12ms/step - loss: 0.0826
Epoch 13/100
6/6 [==============================]